**create sqlalchemy engine**

In [ ]:

from sqlalchemy import URL
from sqlalchemy import create_engine


url_object = URL.create(
    "postgresql+psycopg",
    username="ping",
    password="pink",   # FIXME
    host="localhost",
    database="finance",
)

engine = create_engine(url_object)

**read SP500 components info from csv**

In [39]:
from pathlib import Path

import pandas as pd


file_path = Path('./index/sp500.csv')
assert file_path.exists()

sp500 = pd.read_csv(file_path)
sp500.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [40]:
sp500.columns

Index(['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry',
       'Headquarters Location', 'Date added', 'CIK', 'Founded'],
      dtype='object')

change all column names to lower case

**use lower case for column name to be consistent with db**

In [41]:
sp500.columns = sp500.columns.str.lower()
sp500.head()

,symbol,security,gics sector,gics sub-industry,headquarters location,date added,cik,founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


**store sp500 info to db**

In [ ]:
from sqlalchemy import types

# sp500.to_sql(name='sp500', con=engine,
#              dtype={'symbol': types.VARCHAR, 'security': types.VARCHAR, 'gics sector': types.VARCHAR,
#                     'gics sub-industry': types.VARCHAR, 'headquarters location': types.VARCHAR, 
#                     'date added': types.DATE, 'cik': types.INTEGER, 'founded': types.VARCHAR})

-1

**verify table**

In [43]:
from sqlalchemy import text

with engine.connect() as conn:
    rows = conn.execute(text("SELECT * FROM sp500")).fetchmany(5)
    for row in rows:
        print(row)

(0, 'MMM', '3M', 'Industrials', 'Industrial Conglomerates', 'Saint Paul, Minnesota', datetime.date(1957, 3, 4), 66740, '1902')
(1, 'AOS', 'A. O. Smith', 'Industrials', 'Building Products', 'Milwaukee, Wisconsin', datetime.date(2017, 7, 26), 91142, '1916')
(2, 'ABT', 'Abbott Laboratories', 'Health Care', 'Health Care Equipment', 'North Chicago, Illinois', datetime.date(1957, 3, 4), 1800, '1888')
(3, 'ABBV', 'AbbVie', 'Health Care', 'Biotechnology', 'North Chicago, Illinois', datetime.date(2012, 12, 31), 1551152, '2013 (1888)')
(4, 'ACN', 'Accenture', 'Information Technology', 'IT Consulting & Other Services', 'Dublin, Ireland', datetime.date(2011, 7, 6), 1467373, '1989')


**download SP500 stock data using yfinance**

In [44]:
from src.utils import sp500_symbols

symbols = sp500_symbols()
print(f'there are {len(symbols)} stocks in SP500 list.')

there are 503 stocks in SP500 list.


In [45]:
print(f'some SP500 stock symbol: {symbols[:5]}')

some SP500 stock symbol: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN']


In [46]:
import yfinance as yf


period = 'max'  # '5d'  '1y'  'max'

stocks_df = yf.download(tickers=symbols, group_by='Ticker', period=period)

[*********************100%***********************]  503 of 503 completed


In [51]:
assert stocks_df is not None
stocks_df.head()

Ticker      KHC                       AKAM                        ...  CAG  \
Price      Open High Low Close Volume Open High Low Close Volume  ... Open   
Date                                                              ...        
1962-01-02  NaN  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  ...  NaN   
1962-01-03  NaN  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  ...  NaN   
1962-01-04  NaN  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  ...  NaN   
1962-01-05  NaN  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  ...  NaN   
1962-01-08  NaN  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  ...  NaN   

Ticker                            MDT                        
Price      High Low Close Volume Open High Low Close Volume  
Date                                                         
1962-01-02  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  
1962-01-03  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  
1962-01-04  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  
1962-01-05  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  
1962-01-08  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  

[5 rows x 2515 columns]

In [52]:
assert stocks_df is not None
stocks_df.columns

MultiIndex([( 'KHC',   'Open'),
            ( 'KHC',   'High'),
            ( 'KHC',    'Low'),
            ( 'KHC',  'Close'),
            ( 'KHC', 'Volume'),
            ('AKAM',   'Open'),
            ('AKAM',   'High'),
            ('AKAM',    'Low'),
            ('AKAM',  'Close'),
            ('AKAM', 'Volume'),
            ...
            ( 'CAG',   'Open'),
            ( 'CAG',   'High'),
            ( 'CAG',    'Low'),
            ( 'CAG',  'Close'),
            ( 'CAG', 'Volume'),
            ( 'MDT',   'Open'),
            ( 'MDT',   'High'),
            ( 'MDT',    'Low'),
            ( 'MDT',  'Close'),
            ( 'MDT', 'Volume')],
           names=['Ticker', 'Price'], length=2515)

**organize the multi-level column names**

In [53]:
assert stocks_df is not None
transformed_df = stocks_df.stack(level=0, future_stack=True).rename_axis(['Date', 'Ticker']).reset_index(level=1)

In [54]:
transformed_df.head()

Price,Ticker,Open,High,Low,Close,Volume
Date,,,,,,
1962-01-02,KHC,NaN,NaN,NaN,NaN,NaN
1962-01-02,AKAM,NaN,NaN,NaN,NaN,NaN
1962-01-02,BG,NaN,NaN,NaN,NaN,NaN
1962-01-02,TGT,NaN,NaN,NaN,NaN,NaN
1962-01-02,BKNG,NaN,NaN,NaN,NaN,NaN


In [56]:
print(f'there are {transformed_df.shape[0]:,} rows of stock records.')

there are 8,018,323 rows of stock records.


In [57]:
df_valid = transformed_df.dropna()
print(f'there are {df_valid.shape[0]:,} valid records.')

there are 4,305,734 valid records.


In [59]:
df_valid.head()

Price,Ticker,Open,High,Low,Close,Volume
Date,,,,,,
1962-01-02,BA,0.194272,0.194272,0.190931,0.190931,352350.0
1962-01-02,DTE,0.000000,0.403625,0.399514,0.399514,1880.0
1962-01-02,XOM,0.000000,0.092485,0.091803,0.091803,902400.0
1962-01-02,KR,0.000000,0.033329,0.032100,0.032510,153600.0
1962-01-02,CNP,0.000000,0.292888,0.290679,0.290679,13879.0


In [60]:
df_valid.tail()

Price,Ticker,Open,High,Low,Close,Volume
Date,,,,,,
2025-05-02,TXT,71.410004,72.059998,69.809998,71.019997,2105600.0
2025-05-02,GOOG,164.955002,166.699997,163.660004,165.809998,16832500.0
2025-05-02,RF,20.879999,21.100000,20.760000,21.049999,5370300.0
2025-05-02,CAG,24.250000,24.299999,23.670000,23.860001,5309400.0
2025-05-02,MDT,84.480003,85.070000,83.879997,84.849998,5727500.0


In [61]:
df_valid.describe()

Price,Open,High,Low,Close,Volume
count,4.305734e+06,4.305734e+06,4.305734e+06,4.305734e+06,4.305734e+06
mean,4.968712e+01,5.035539e+01,4.917392e+01,4.977768e+01,6.380533e+06
std,1.571611e+02,1.590132e+02,1.552989e+02,1.571711e+02,4.028414e+07
min,0.000000e+00,1.482385e-03,1.482384e-03,1.509666e-03,0.000000e+00
25%,4.195714e+00,4.345333e+00,4.224806e+00,4.287446e+00,4.972000e+05
50%,1.729784e+01,1.754895e+01,1.707008e+01,1.731531e+01,1.479200e+06
75%,4.792895e+01,4.846486e+01,4.737832e+01,4.794156e+01,3.926000e+06
max,9.914170e+03,9.964770e+03,9.794000e+03,9.924400e+03,9.230856e+09


**clean the df**

In [62]:
df_valid['Volume'] = df_valid['Volume'].fillna(0).astype(int)

for c in ['Open', 'High', 'Low', 'Close']:
    df_valid[c] = df_valid[c].fillna(0).round(2)

df_valid.head()

/tmp/ipykernel_8853/866933911.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid['Volume'] = df_valid['Volume'].fillna(0).astype(int)
/tmp/ipykernel_8853/866933911.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid[c] = df_valid[c].fillna(0).round(2)


Price,Ticker,Open,High,Low,Close,Volume
Date,,,,,,
1962-01-02,BA,0.19,0.19,0.19,0.19,352350
1962-01-02,DTE,0.00,0.40,0.40,0.40,1880
1962-01-02,XOM,0.00,0.09,0.09,0.09,902400
1962-01-02,KR,0.00,0.03,0.03,0.03,153600
1962-01-02,CNP,0.00,0.29,0.29,0.29,13879


**set 'Date' as a column**

In [63]:
df_valid.reset_index(inplace=True)
df_valid.head()

Price,Date,Ticker,Open,High,Low,Close,Volume
0,1962-01-02,BA,0.19,0.19,0.19,0.19,352350
1,1962-01-02,DTE,0.00,0.40,0.40,0.40,1880
2,1962-01-02,XOM,0.00,0.09,0.09,0.09,902400
3,1962-01-02,KR,0.00,0.03,0.03,0.03,153600
4,1962-01-02,CNP,0.00,0.29,0.29,0.29,13879


In [64]:
df_valid.columns = df_valid.columns.str.lower()
df_valid.head()

Price,date,ticker,open,high,low,close,volume
0,1962-01-02,BA,0.19,0.19,0.19,0.19,352350
1,1962-01-02,DTE,0.00,0.40,0.40,0.40,1880
2,1962-01-02,XOM,0.00,0.09,0.09,0.09,902400
3,1962-01-02,KR,0.00,0.03,0.03,0.03,153600
4,1962-01-02,CNP,0.00,0.29,0.29,0.29,13879


In [ ]:
from sqlalchemy.orm import declarative_base
from sqlalchemy import UniqueConstraint, Column, Integer, String, Date, Numeric, BigInteger


Base = declarative_base()

class StockPrice(Base):
    __tablename__ = 'stock_prices'
    __table_args__ = (UniqueConstraint('ticker', 'date'),)  # tuple

    id = Column(Integer, primary_key=True)
    ticker = Column(String(10), nullable=False)
    date = Column(Date, nullable=False)
    open = Column(Numeric(10, 2))
    high = Column(Numeric(10, 2))
    low = Column(Numeric(10, 2))
    close = Column(Numeric(10, 2))
    volume = Column(BigInteger)  # max volume is more than 9B


Base.metadata.create_all(engine)


In [66]:
column_names = [column.name for column in StockPrice.__table__.columns]
print(column_names)

['id', 'ticker', 'date', 'open', 'high', 'low', 'close', 'volume']


In [67]:
df_valid.columns.to_list()

['date', 'ticker', 'open', 'high', 'low', 'close', 'volume']

In [68]:
df_valid.to_sql(name='stock_prices', con=engine, if_exists='append', index=False, chunksize=10_000)

-431

In [70]:
from sqlalchemy.orm import Session

session = Session(engine)

n_rows = session.query(StockPrice).count()

assert df_valid.shape[0] == n_rows
print(f'There are {n_rows:,} rows in table {StockPrice.__tablename__}')

There are 4,305,734 rows in table stock_prices
